# Tutorial 17: Riskfolio-Lib with MOSEK for Real Applications (612 assets and 4943 observations)

이 튜토리얼은 대규모 문제를 다룰 때 Riskfolio-Lib를 실제 응용 프로그램에서 사용하는 방법을 보여줍니다. 이는 은행, 헤지 펀드, 뮤추얼 펀드 및 기타 기관 투자자들이 일상적으로 직면하는 상황입니다.

## 1. Downloading the data:

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import warnings

warnings.filterwarnings("ignore")
pd.options.display.float_format = '{:.4%}'.format

data = pd.read_csv("assets_data.csv", index_col='Dates')

Y = data.pct_change().dropna()

display(Y.head())

,A,AAPL,ABC,ABMD,ABT,ADBE,ADM,ADSK,AEE,AEP,...,WWD,X,Y,ZBRA,ASML,BMRN,CHKP,MXIM,SHPG,SIRI
Dates,,,,,,,,,,,,,,,,,,,,,
04-01-2000,-8.3842%,-8.4321%,-6.8261%,-2.3945%,-1.9714%,-8.3890%,-1.0419%,-4.8695%,0.0000%,1.1928%,...,0.4755%,-3.7328%,-0.5362%,-1.4428%,-6.6594%,6.3725%,-9.0390%,-3.6943%,-5.6250%,-6.6869%
05-01-2000,-4.7666%,1.4642%,7.7584%,1.0498%,-0.7307%,1.9773%,-1.5792%,-6.2992%,3.8685%,3.7328%,...,-2.8302%,1.0204%,-0.3703%,1.9143%,-2.7618%,-5.0691%,-1.0062%,0.2646%,5.5188%,5.0489%
06-01-2000,-5.7057%,-8.6530%,7.6008%,0.1722%,1.6572%,0.8163%,0.5349%,-6.0921%,-0.3724%,0.5682%,...,-1.2140%,0.6061%,0.5404%,-5.4143%,-6.1538%,-0.9709%,-6.8694%,2.7704%,5.6485%,2.9457%
07-01-2000,10.4035%,4.7364%,13.0097%,-0.5158%,3.9857%,4.8583%,1.5960%,11.8564%,1.4953%,1.3183%,...,0.2453%,2.6104%,2.7229%,-1.1099%,2.2697%,4.9020%,4.7156%,2.3107%,8.9109%,-1.5060%
10-01-2000,6.0575%,-1.7588%,2.6318%,14.2825%,-2.2653%,3.8610%,0.0000%,-1.1994%,-0.7366%,-0.3717%,...,-6.1271%,-0.7828%,-1.5378%,3.3669%,9.4945%,2.3364%,10.3583%,1.3802%,-0.7273%,3.9755%


## 2. Estimating Max Mean/Risk Portfolios for all Risk Measures

In [2]:
import riskfolio as rp

# Building the portfolio object
port = rp.Portfolio(returns=Y)

# Calculating optimum portfolio

# Select method and estimate input parameters:

method_mu='hist' # Method to estimate expected returns based on historical data.
method_cov='hist' # Method to estimate covariance matrix based on historical data.

port.assets_stats(method_mu=method_mu, method_cov=method_cov)

# Input model parameters:

port.solvers = ['MOSEK'] # Setting MOSEK as the default solver
# if you want to set some MOSEK params use this code as an example
# import mosek
# port.sol_params = {'MOSEK': {'mosek_params': {mosek.iparam.intpnt_solve_form: mosek.solveform.dual}}}

port.alpha = 0.05 # Significance level for CVaR, EVaR y CDaR 
model='Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
obj = 'Sharpe' # Objective function, could be MinRisk, MaxRet, Utility or Sharpe
hist = True # Use historical scenarios for risk measures that depend on scenarios
rf = 0 # Risk free rate
l = 0 # Risk aversion factor, only useful when obj is 'Utility'

Matplotlib is building the font cache; this may take a moment.


### 2.1 Optimizing Process by Risk Measure

In [3]:
from timeit import default_timer as timer
from datetime import timedelta

# Risk Measures available:
#
# 'MV': Standard Deviation.
# 'MAD': Mean Absolute Deviation.
# 'GMD': Gini Mean Difference.
# 'MSV': Semi Standard Deviation.
# 'FLPM': First Lower Partial Moment (Omega Ratio).
# 'SLPM': Second Lower Partial Moment (Sortino Ratio).
# 'CVaR': Conditional Value at Risk.
# 'TG': Tail Gini.
# 'EVaR': Entropic Value at Risk.
# 'RLVaR': Relativistic Value at Risk.
# 'WR': Worst Realization (Minimax)
# 'CVRG': CVaR Range.
# 'TGRG': Tail Gini Range.
# 'RG': Range
# 'ADD': Average Drawdown of uncompounded cumulative returns.
# 'UCI': Ulcer Index of uncompounded cumulative returns.
# 'CDaR': Conditional Drawdown at Risk of uncompounded cumulative returns.
# 'EDaR': Entropic Drawdown at Risk of uncompounded cumulative returns.
# 'RLDaR': Relativistic Drawdown at Risk of uncompounded cumulative returns.
# 'MDD': Maximum Drawdown of uncompounded cumulative returns (Calmar Ratio).

# rms = ["MV", "MAD", "GMD", "MSV", "FLPM", "SLPM",
#        "CVaR", "TG", "EVaR", "RLVaR", "WR", "CVRG",
#        "TGRG", "RG", "ADD", "UCI", "CDaR", "EDaR",
#        "RLDaR", "MDD"]

rms = ["MV", "MAD", "MSV", "FLPM", "SLPM",
       "CVaR", "EVaR", "RLVaR", "WR", 
       "ADD", "UCI", "CDaR", "EDaR",
       "RLDaR", "MDD"]

w = {}
for rm in rms:
    start = timer()
    w[rm] = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)
    end = timer()
    print(rm + ' takes ',timedelta(seconds=end-start))

MV takes  0:00:00.652999
MAD takes  0:00:02.814170
MSV takes  0:00:06.354396
FLPM takes  0:00:02.604594
SLPM takes  0:00:22.226664
CVaR takes  0:00:02.984764
EVaR takes  0:00:46.870661
RLVaR takes  0:00:42.702724
WR takes  0:00:02.357783
ADD takes  0:00:09.483642
UCI takes  0:00:07.117627
CDaR takes  0:00:09.593431
EDaR takes  0:00:09.081312
RLDaR takes  0:00:12.746258
MDD takes  0:00:31.153549


### 2.2 Portfolio Weights

In [4]:
w_s = pd.DataFrame([])
for rm in rms:
    w_s = pd.concat([w_s, w[rm]], axis=1)

w_s.columns = rms

display(w_s)

,MV,MAD,MSV,FLPM,SLPM,CVaR,EVaR,RLVaR,WR,ADD,UCI,CDaR,EDaR,RLDaR,MDD
A,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
AAPL,2.7531%,2.3440%,2.8935%,2.0113%,2.9581%,3.7962%,4.9729%,0.5676%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
ABC,2.3723%,0.7081%,3.6590%,1.1216%,3.8779%,2.9236%,10.5781%,1.8319%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
ABMD,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.3634%,0.0000%,0.0000%,0.0000%,0.0000%
ABT,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BMRN,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
CHKP,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
MXIM,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
SHPG,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%


### 2.3 In sample CAGR by Portfolio

In [5]:
from datetime import datetime
a1 = datetime.strptime(data.index[0], '%d-%m-%Y')
a2 = datetime.strptime(data.index[-1], '%d-%m-%Y')
days = (a2-a1).days

cagr = {} 
for rm in rms:
    a = np.prod(1 + Y @ w_s[rm]) ** (360/days)-1
    cagr[rm] = [a]

cagr = pd.DataFrame(cagr).T
cagr.columns = ['CAGR']

cagr.style.format("{:.2%}").background_gradient(cmap='RdYlGn')

,CAGR
MV,28.20%
MAD,28.01%
MSV,28.65%
FLPM,28.07%
SLPM,28.68%
CVaR,27.91%
EVaR,29.53%
RLVaR,28.27%
WR,26.71%
ADD,25.91%


## 3. Estimating Min Risk Portfolios for all Risk Measures

### 3.1 Optimizing Process by Risk Measure

In [6]:
# rms = ["MV", "MAD", "GMD", "MSV", "FLPM", "SLPM",
#        "CVaR", "TG", "EVaR", "RLVaR", "WR", "CVRG",
#        "TGRG", "RG", "ADD", "UCI", "CDaR", "EDaR",
#        "RLDaR", "MDD"]

rms = ["MV", "MAD", "MSV", "FLPM", "SLPM",
       "CVaR", "EVaR", "RLVaR", "WR", 
       "ADD", "UCI", "CDaR", "EDaR",
       "RLDaR", "MDD"]

w_min = {}
obj = 'MinRisk'
for rm in rms:
    start = timer()
    w_min[rm] = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)
    end = timer()
    print(rm + ' takes ',timedelta(seconds=end-start))

MV takes  0:00:00.538174
MAD takes  0:00:02.539509
MSV takes  0:00:03.560080
FLPM takes  0:00:02.315277
SLPM takes  0:00:10.991407
CVaR takes  0:00:02.560665
EVaR takes  0:00:30.388114
RLVaR takes  0:00:32.432976
WR takes  0:00:02.233095
ADD takes  0:00:08.415913
UCI takes  0:00:06.877365
CDaR takes  0:00:10.243082
EDaR takes  0:00:04.654421
RLDaR takes  0:00:05.903299
MDD takes  0:00:08.486983


### 3.2 Portfolio Weights

In [7]:
w_min_s = pd.DataFrame([])
for rm in rms:
    w_min_s = pd.concat([w_min_s, w_min[rm]], axis=1)

w_min_s.columns = rms

display(w_min_s)

,MV,MAD,MSV,FLPM,SLPM,CVaR,EVaR,RLVaR,WR,ADD,UCI,CDaR,EDaR,RLDaR,MDD
A,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
AAPL,0.0000%,0.1746%,0.0000%,0.5335%,0.0000%,0.0000%,2.3313%,2.7067%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
ABC,0.0000%,0.0000%,0.0001%,0.1824%,0.0743%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
ABMD,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
ABT,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BMRN,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
CHKP,0.0000%,0.0000%,0.0002%,0.0000%,0.0003%,0.0000%,0.0000%,0.0000%,0.0000%,3.1074%,3.6153%,4.2516%,0.0000%,0.0000%,0.0000%
MXIM,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,1.9335%,4.6054%,4.4013%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%
SHPG,0.0000%,0.8500%,0.0000%,0.4403%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%,0.0000%


### 3.3 In sample CAGR by Portfolio

In [8]:
from datetime import datetime
a1 = datetime.strptime(data.index[0], '%d-%m-%Y')
a2 = datetime.strptime(data.index[-1], '%d-%m-%Y')
days = (a2-a1).days

min_cagr = {} 
for rm in rms:
    a = np.prod(1 + Y @ w_min_s[rm]) ** (360/days)-1
    min_cagr[rm] = [a]

min_cagr = pd.DataFrame(min_cagr).T
min_cagr.columns = ['CAGR']

min_cagr.style.format("{:.2%}").background_gradient(cmap='RdYlGn')

,CAGR
MV,10.79%
MAD,9.87%
MSV,10.43%
FLPM,12.26%
SLPM,11.69%
CVaR,12.30%
EVaR,10.52%
RLVaR,10.75%
WR,9.98%
ADD,19.02%


## 4. Estimating Risk Parity Portfolios for all Risk Measures

### 4.1 Optimizing Process by Risk Measure

In [9]:
# rms = ["MV", "MAD", "GMD", "MSV", "FLPM", "SLPM",
#        "CVaR", "TG", "EVaR", "RLVaR", "CVRG",
#        "TGRG", "UCI", "CDaR", "EDaR", "RLDaR"]

rms = ["MV", "MAD", "MSV", "FLPM", "SLPM",
       "CVaR", "EVaR", "RLVaR", "CVRG",
       "UCI", "CDaR", "EDaR", "RLDaR"]
b = None # Risk contribution constraints vector, when None is equally risk per asset

w_rp = {}
for rm in rms:
    start = timer()
    

    w_rp[rm] = port.rp_optimization(model=model, rm=rm, rf=rf, b=b, hist=hist)
    end = timer()
    print(rm + ' takes ',timedelta(seconds=end-start))

MV takes  0:00:00.966015
MAD takes  0:00:03.564861
MSV takes  0:00:03.979257
FLPM takes  0:00:07.376031
SLPM takes  0:00:12.473310
CVaR takes  0:00:11.634557
EVaR takes  0:00:31.562762
RLVaR takes  0:00:48.941637


### 4.2 Portfolio Weights

In [ ]:
w_rp_s = pd.DataFrame([])
for rm in rms:
    w_rp_s = pd.concat([w_rp_s, w_rp[rm]], axis=1)

w_rp_s.columns = rms

display(w_rp_s)

### 4.3 In sample CAGR

In [ ]:
rp_cagr = {} 
for rm in rms:
    a = np.prod(1 + Y @ w_rp_s[rm]) ** (360/days)-1
    rp_cagr[rm] = [a]

rp_cagr = pd.DataFrame(rp_cagr).T
rp_cagr.columns = ['CAGR']

rp_cagr.style.format("{:.2%}").background_gradient(cmap='RdYlGn')